# 비지도학습 군집화 테스트

###  "사용 데이터 : 장르별 베스트셀러 리뷰 모음"

### 조건
    * 각각의 ISBN끼리 리뷰를 합쳐서 테스트
    * 명사
    * topics : 6
    * 테스트 : 20

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

%matplotlib inline

import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import urllib

In [2]:
import konlpy
from konlpy.tag import Okt

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


# 데이터 전처리

## 도서 리뷰 추가

In [4]:
import glob
import pandas as pd
from pandas.core.dtypes.missing import notnull
import nltk
from konlpy.tag import Okt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
import re
from gensim.models import Word2Vec
from pandas import DataFrame
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## <전처리>

In [5]:
def cleansing(text):
    # text = str(text)
    pattern = "([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)"  # e-mail 주소 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+"  # url 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "([^ㄷㅋㅎㅜㅠ가-힣a-zA-Z0-9])+"  # 한글 자음, 모음 제거 - without 'ㄷ,ㅋ,ㅎ,ㅜ,ㅠ'
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "<[^>]*>"  # html tag 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "[\r|\n]"  # \r, \n 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = re.compile(r"\s+")  # 이중 space 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    return text

### ISBN을 기준으로 리뷰를 모은 뒤 재조합

## <토큰화>

#### 분류 테스트 

In [6]:
okt = Okt()

----

### 기존 데이터 가져오기

In [7]:
# 불러오기
import pickle

book_morphs_with_isbn_pos_non_stopword = []
with open('data/detail_bestbook_unit_isbn_with_isbn_pos_non_stopword.txt', 'rb') as lf:
    book_morphs_with_isbn_pos_non_stopword = pickle.load(lf)
    
print(book_morphs_with_isbn_pos_non_stopword[0])

[9788959752195, [('내', 'Noun'), ('가제', 'Noun'), ('일', 'Noun'), ('좋아하다', 'Adjective'), ('책', 'Noun'), ('이', 'Noun'), ('당', 'Noun'), ('6', 'Number')]]


In [8]:
print(len(book_morphs_with_isbn_pos_non_stopword))

print(book_morphs_with_isbn_pos_non_stopword[2])

42559
[9791127420949, [('덕분', 'Noun'), ('에', 'Josa'), ('귀엽다', 'Adjective'), ('SD', 'Alpha'), ('를', 'Noun'), ('그릴', 'Noun'), ('수', 'Noun'), ('있다', 'Adjective'), ('되어다', 'Verb'), ('너무', 'Adverb'), ('기쁘다', 'Adjective'), ('별', 'Noun'), ('5', 'Number'), ('개', 'Noun'), ('로', 'Noun'), ('도', 'Noun'), ('사실', 'Noun'), ('모자라다', 'Verb'), ('정도', 'Noun'), ('로', 'Noun'), ('좋다', 'Adjective'), ('책', 'Noun'), ('채색', 'Noun'), ('이', 'Noun'), ('없다', 'Adjective'), ('게', 'Noun'), ('좀', 'Noun'), ('아쉽다', 'Adjective'), ('채색', 'Noun'), ('책', 'Noun'), ('이', 'Noun'), ('아니다', 'Adjective'), ('별', 'Noun'), ('기대', 'Noun'), ('안', 'Noun'), ('하다', 'Verb'), ('생각', 'Noun'), ('보다', 'Verb'), ('내용', 'Noun'), ('이', 'Noun'), ('알차다', 'Adjective'), ('두고두고', 'Noun'), ('볼', 'Noun'), ('것', 'Noun'), ('같다', 'Adjective')]]


In [9]:
print(book_morphs_with_isbn_pos_non_stopword[0])

[9788959752195, [('내', 'Noun'), ('가제', 'Noun'), ('일', 'Noun'), ('좋아하다', 'Adjective'), ('책', 'Noun'), ('이', 'Noun'), ('당', 'Noun'), ('6', 'Number')]]


In [10]:
#  도서 불용어를 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','것','라고','에게','라면','을','이라',
             '라니','있다','아','랑','쯤된','에서','에선','어','이지만','으로나','때','때는','때라면','때라서','라','이다','있','죠','고','니','로','있','같',
             '어서','어요','는데','습니다','면서','많이','마','더','그렇다','의','당','좀','책','안','볼','게','안','정말','듯','이제야','여','요','게다가','같다',
            '임','로서','이제','만','인','붙이','그','저','수','가제','부터','닷','저희','적','알','쉬','못','꼭','살','제','권','제','분','나','내', '진작', '전', '뿐'
            , '대한','대해']

In [11]:
######### 태깅한 데이터 중에서 명사 + 형용사(1이상) + 불용어 처리

book_train_nouns = []
# book_train_na_with_isbn = []
for (isbn, test) in book_morphs_with_isbn_pos_non_stopword :
    tts = [(word,pos) for (word,pos) in test if not word in stopwords] #불용어 처리
    na = [word for (word,pos) in tts if pos in 'Noun'] # 명사만 저장

    if na : 
        book_train_nouns.append(na)
#         book_train_na_with_isbn.append([isbn,na])

print('**********************END***************************')

**********************END***************************


In [13]:
for i in range(0,5) :
#     print(book_morphs_with_isbn_pos_non_stopword[i])
    print(book_train_nouns[i])

['일']
['처음', '제목', '보고', '예상', '스토리', '완전', '신간', '홀린', '장바구니', '사서', '맘', '사서', '걸', '조금', '여자', '이야기']
['덕분', '그릴', '별', '개', '사실', '정도', '채색', '채색', '별', '기대', '생각', '내용', '두고두고']
['구매', '쭉', '지리', '여행', '여행', '가기']
['요가', '궁극', '의미', '요가', '수행자', '위', '메시지', '방법', '과정', '자세', '설명', '알다', '정리', '매우', '서적', '요가', '공부', '위해', '구입', '공부', '도움', '요가', '교과서', '말', '요가', '백과사전', '식', '명작', '설명', '그림', '첨부', '재미', '실제', '응용', '위', '참고서', '아헹', '선생님', '요가', '알라딘', '쿠폰', '무료', '배송', '보아', '요가', '교과서', '최고', '요가', '입문', '무리']


----

# 정수 인코딩과 단어 집합 만들기

(인코딩값, 빈도수)

#### 데이터 셋
* 명사

In [15]:
from gensim import corpora


dictionary = corpora.Dictionary(book_train_nouns)
corpus = [dictionary.doc2bow(text) for text in book_train_nouns]

In [16]:
print(len(book_train_nouns))
print(len(dictionary))
print(len(corpus))

42337
52744
42337


In [19]:
print(corpus[24698])

[(12, 1), (24, 1), (90, 1), (183, 1), (460, 1), (468, 1), (493, 1), (557, 1), (661, 2), (692, 1), (872, 1), (1064, 1), (1175, 1), (1254, 1), (1432, 1), (1477, 1), (1760, 1), (1919, 1), (1985, 2), (6692, 1), (9070, 1), (12701, 1)]


In [20]:
print(dictionary[1985])
print(dictionary[661])

라틴어
뜻


# LDA 모델 훈련시키기


passes는 알고리즘의 동작 횟수

In [26]:
import gensim

NUM_TOPICS = 6
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

d:\project02\field-team4_test\machine_learning\web_scraping\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.016*"생각" + 0.010*"우리" + 0.010*"역사" + 0.009*"사람"')
(1, '0.020*"이야기" + 0.019*"작가" + 0.018*"소설" + 0.012*"작품"')
(2, '0.085*"아이" + 0.038*"그림" + 0.018*"여행" + 0.012*"엄마"')
(3, '0.017*"구매" + 0.015*"번역" + 0.014*"내용" + 0.012*"만화"')
(4, '0.035*"공부" + 0.017*"영어" + 0.016*"도움" + 0.014*"문제"')
(5, '0.020*"생각" + 0.018*"마음" + 0.015*"사람" + 0.009*"말"')


## 시각화


In [27]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

d:\project02\field-team4_test\machine_learning\web_scraping\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==10:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.59129125), (1, 0.10344961), (2, 0.10280105), (3, 0.101016976), (4, 0.10144113)]
1 번째 문서의 topic 비율은 [(0, 0.011877647), (1, 0.42594293), (2, 0.011972798), (3, 0.011934572), (4, 0.538272)]
2 번째 문서의 topic 비율은 [(0, 0.014742933), (1, 0.01459694), (2, 0.014549915), (3, 0.014560348), (4, 0.9415499)]
3 번째 문서의 topic 비율은 [(0, 0.028891737), (1, 0.028688028), (2, 0.028759224), (3, 0.028947055), (4, 0.88471395)]
4 번째 문서의 topic 비율은 [(0, 0.12630336), (3, 0.40253255), (4, 0.4630083)]
5 번째 문서의 topic 비율은 [(0, 0.21212205), (1, 0.034367517), (2, 0.033743933), (3, 0.4272561), (4, 0.29251036)]
6 번째 문서의 topic 비율은 [(0, 0.27039808), (1, 0.011371463), (2, 0.2832794), (3, 0.42363194), (4, 0.011319142)]
7 번째 문서의 topic 비율은 [(0, 0.19629492), (1, 0.6541123), (4, 0.13432786)]
8 번째 문서의 topic 비율은 [(0, 0.10099205), (1, 0.100991845), (2, 0.5960278), (3, 0.100994594), (4, 0.100993715)]
9 번째 문서의 topic 비율은 [(0, 0.020428438), (1, 0.020821277), (2, 0.918062), (3, 0.020043312), (4, 0.020645026)]


d:\project02\field-team4_test\machine_learning\web_scraping\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
